In [2]:
%pylab
%matplotlib inline
from PIL import Image
import numpy as np

Using matplotlib backend: <object object at 0x000001AA00022D70>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Funktion, die die LAB-Werte richtig ausrechnet (LAB Werte können im Vergleich zu RGB und HSV auch negativ sein!)

In [3]:
# Python3 program change RGB Color
# Model to LAB Color Model


def rgb_to_lab(r, g, b):
    RGB = [r,g,b]
    num = 0
    for value in RGB:
        value = float(value)/255
        if value > 0.04045:
            value = ((value + 0.055) / 1.055) ** 2.4
        else:
            value = value / 12.92
        
        RGB[num] = value *100
        num = num + 1
    
    XYZ = [0,0,0]

    X = RGB[0] * 0.4124 + RGB[1] * 0.3576 + RGB[2] * 0.1805
    Y = RGB[0] * 0.2126 + RGB[1] * 0.7152 + RGB[2] * 0.0722
    Z = RGB[0] * 0.0193 + RGB[1] * 0.1192 + RGB[2] * 0.9505

    XYZ[0] = round(X, 4)
    XYZ[1] = round(Y, 4)
    XYZ[2] = round(Z, 4)

    XYZ[0] = float(XYZ[0]) / 95.047         # ref_X =  95.047   Observer= 2°, Illuminant= D65
    XYZ[1] = float(XYZ[1]) / 100.0          # ref_Y = 100.000
    XYZ[2] = float(XYZ[2]) / 108.883        # ref_Z = 108.883

    num = 0
    for value in XYZ :

       if value > 0.008856 :
           value = value ** (1/3)
       else :
           value = ( 7.787 * value ) + ( 16 / 116 )

       XYZ[num] = value
       num = num + 1

  

    L = (116 * XYZ[1]) - 16
    A = 500 * (XYZ[0] - XYZ[1])
    B = 200 * (XYZ[1] - XYZ[2])

    return L, A, B

rgb_to_lab(20, 200, 80)

(70.89607739403526, -65.75030489750277, 47.6334090693872)

Fancy Funktion für die RGB-to-LAB Konversion
Erklärung der Fancy Funktion:
- isistance: Bild muss als PIL Image eingelesen werden (open Funktion)
- RGB channels werden durch split Methode in 3 Banden (r, g und b) gesplittet
- leere Listen werden erstellt
- rd (von red), gn (green), bl (blue) 
- zip Methode erstellt Tupeln von den R, G und B Banden --> getdata extrahiert die Werte der jeweiligen Banden (R, G, B) --> so werden die RGB Werte für jedes Pixel in ein Tupel reingepackt
- Diese RGB Werte werden zu LAB konvertiert
- Die Werte der R, G und B Banden werden durch die LAB-Werte ersetzt. Achtung! Es bleiben trotzdem als r, g, b benannt, weil die 3 Banden am Anfang so definiert wurden
- Deshalb wird bei merge Funktion r, g, b eingegeben, aber Mode muss LAB sein

NACHTEIL: das Bild ist aber zu blau (Negative Werte werden zu 0, vermutlich putdata-Methode ist fehlerhaft
VORTEIL: aber genau so scharf wie Input-Bild. Output-Bild funktioniert auch gut auf dem K-Means Clustering Algorithmus


In [4]:
def LABColor(img):
    if isinstance(img, Image.Image):
        r,g,b = img.split()
        Ldat = []
        Adat = []
        Bdat = []
        for rd, gn, bl in zip(r.getdata(), g.getdata(), b.getdata()):
            L,A,B = rgb_to_lab(rd, gn, bl)
            Ldat.append(L)
            Adat.append(A)
            Bdat.append(B)
        r.putdata(Ldat)
        g.putdata(Adat)
        b.putdata(Bdat)
        return Image.merge("LAB", (r,g,b))
    else:
        return None


In [5]:
RGB_image = Image.open("../data/Yeast_Cells.jpg")
RGB_array = np.array(RGB_image)
LAB_image_fancy = LABColor(RGB_image)
LAB_image_fancy.save("../data/LAB_YeastCells_fancy.tiff")

Nicht so fancy Funktion (aber vielleicht verständlicher) mit array, das Output-Bild ist normal was die Farben angeht, aber an manchen Stellen verpixelter als die Fancy Funktion
Erklärung Funktion mit array:
- Bild wird in array umgewandelt
- Leere Liste wird erstellt
- für jedes Pixel werden die RGB Werte aus dem RGB_array in LAB Werte umgewandelt
- LAB Werte werden für jedes Pixel der Liste eingefügt
- Liste mit LAB Werten wird zu 3D-Array mit den gleichem Shape wie das Bild umgewandelt

NACHTEIL: Bild nicht so scharf (Information geht irgendwie verloren); Bild auch bisschen dunkler; Output von K Means ganz blöd
VORTEIL: nicht blau, Farben sind generell normal

In [6]:
def image_lab_conversion_array(RGB_image):
    width, height = RGB_image.size
    RGB_array = np.array(RGB_image)
    LAB_list = []
    for x in range(width):
        for y in range(height):
            r, g, b = RGB_array[x,y]
            L, A, B = rgb_to_lab(r, g, b)
            listLAB = [int(L), int(A), int(B)]
            LAB_list.append(listLAB)
    LAB_array = np.asarray(LAB_list, dtype=uint8).reshape(width, height,3)
    return LAB_array



LAB-Bilder können nicht direkt in Python angezeigt werden, sie müssen direkt im TIFF Format (andere Formate anscheinend nicht unterstützt) gespeichert werden!

In [7]:
#no need to run, Bild ist schon gespeichert und gepusht
LAB_array = image_lab_conversion_array(RGB_image)
LAB_image_from_array = Image.fromarray(LAB_array, "LAB").save("../data/LAB_YeastCells_from_array.tiff")

Ihr könnt euch im Data Folder die LAB Bilder anschauen und sie mit den normalen Bildern vergleichen!